In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import string
import re
import emoji

In [2]:
def load_data():
    data = pd.read_csv('anies.csv')
    return data
df = load_data()
df.tail()

Unnamed: 0                   id                 date  \
2560        2383  1686005950476677120  2023-07-31 13:28:37   
2561        2386  1686005857828679681  2023-07-31 13:28:15   
2562        2391  1686005757882593280  2023-07-31 13:27:51   
2563        2393  1686005743378731008  2023-07-31 13:27:48   
2564        2394  1686005736676171776  2023-07-31 13:27:46   

                                                   text  \
2560  RT @rahmaniarbaftim: Untuk bisa menjadi Pemimp...   
2561  RT @YepryR: Terimakasih masyarakat brebes jawa...   
2562  Anies Baswedan tegaskan tidak akan lanjutkan P...   
2563  RT @MissKoral: Petani Brebes Jateng Deklarasi ...   
2564  RT @OposisiCerdas: Partai Masyumi Dukung Anies...   

                       place      screen_name  
2560                     NaN  Joewija45450015  
2561                     NaN    Ayong32516903  
2562                     NaN       bhozsadolf  
2563  DKI Jakarta, Indonesia      aly_benzema  
2564                     NaN       muawanah05

In [3]:
df = pd.DataFrame(df[['id', 'date', 'place', 'screen_name', 'text']])
df

id                 date                   place  \
0     1661196530127351808  2023-05-24 02:24:50               INDONESIA   
1     1661196491896266752  2023-05-24 02:24:41                     NaN   
2     1661196455691046912  2023-05-24 02:24:33                     NaN   
3     1661196446203523073  2023-05-24 02:24:30                     NaN   
4     1661196388963856390  2023-05-24 02:24:17                     NaN   
...                   ...                  ...                     ...   
2560  1686005950476677120  2023-07-31 13:28:37                     NaN   
2561  1686005857828679681  2023-07-31 13:28:15                     NaN   
2562  1686005757882593280  2023-07-31 13:27:51                     NaN   
2563  1686005743378731008  2023-07-31 13:27:48  DKI Jakarta, Indonesia   
2564  1686005736676171776  2023-07-31 13:27:46                     NaN   

          screen_name                                               text  
0            jijindut  RT @eddy_widodo_165: Alhamdulilah 🤲\nBacapres ...  
1     georgiussoetjip  RT @renandabachtar: Din Syamsuddin Lebih Senan...  
2        003whitesky1  RT @PutraErlangga95: Ingin menyelamatkan Indon...  
3             et_fath  RT @idtodayco: Kurang Bukti, Relawan Ganjar Pr...  
4             et_fath  RT @Yurissa_Samosir: Sambutan hangat dari Masy...  
...               ...                                                ...  
2560  Joewija45450015  RT @rahmaniarbaftim: Untuk bisa menjadi Pemimp...  
2561    Ayong32516903  RT @YepryR: Terimakasih masyarakat brebes jawa...  
2562       bhozsadolf  Anies Baswedan tegaskan tidak akan lanjutkan P...  
2563      aly_benzema  RT @MissKoral: Petani Brebes Jateng Deklarasi ...  
2564       muawanah05  RT @OposisiCerdas: Partai Masyumi Dukung Anies...  

[2565 rows x 5 columns]

# CLEANSING

In [4]:
# Menghapus nantion username
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# Menghapus @user
df['cleansing'] = np.vectorize(remove_pattern) (df['text'], "@[\w]*")
df

id                 date                   place  \
0     1661196530127351808  2023-05-24 02:24:50               INDONESIA   
1     1661196491896266752  2023-05-24 02:24:41                     NaN   
2     1661196455691046912  2023-05-24 02:24:33                     NaN   
3     1661196446203523073  2023-05-24 02:24:30                     NaN   
4     1661196388963856390  2023-05-24 02:24:17                     NaN   
...                   ...                  ...                     ...   
2560  1686005950476677120  2023-07-31 13:28:37                     NaN   
2561  1686005857828679681  2023-07-31 13:28:15                     NaN   
2562  1686005757882593280  2023-07-31 13:27:51                     NaN   
2563  1686005743378731008  2023-07-31 13:27:48  DKI Jakarta, Indonesia   
2564  1686005736676171776  2023-07-31 13:27:46                     NaN   

          screen_name                                               text  \
0            jijindut  RT @eddy_widodo_165: Alhamdulilah 🤲\nBacapres ...   
1     georgiussoetjip  RT @renandabachtar: Din Syamsuddin Lebih Senan...   
2        003whitesky1  RT @PutraErlangga95: Ingin menyelamatkan Indon...   
3             et_fath  RT @idtodayco: Kurang Bukti, Relawan Ganjar Pr...   
4             et_fath  RT @Yurissa_Samosir: Sambutan hangat dari Masy...   
...               ...                                                ...   
2560  Joewija45450015  RT @rahmaniarbaftim: Untuk bisa menjadi Pemimp...   
2561    Ayong32516903  RT @YepryR: Terimakasih masyarakat brebes jawa...   
2562       bhozsadolf  Anies Baswedan tegaskan tidak akan lanjutkan P...   
2563      aly_benzema  RT @MissKoral: Petani Brebes Jateng Deklarasi ...   
2564       muawanah05  RT @OposisiCerdas: Partai Masyumi Dukung Anies...   

                                              cleansing  
0     RT : Alhamdulilah 🤲\nBacapres \nMenurut Litban...  
1     RT : Din Syamsuddin Lebih Senang AHY Jadi Cawa...  
2     RT : Ingin menyelamatkan Indonesia 🇲🇨 silahkan...  
3     RT : Kurang Bukti, Relawan Ganjar Pranowo Gaga...  
4     RT : Sambutan hangat dari Masyarakat Ponorogo ...  
...                                                 ...  
2560  RT : Untuk bisa menjadi Pemimpin kelas dunia.\...  
2561  RT : Terimakasih masyarakat brebes jawa tengah...  
2562  Anies Baswedan tegaskan tidak akan lanjutkan P...  
2563  RT : Petani Brebes Jateng Deklarasi Dukung Ani...  
2564  RT : Partai Masyumi Dukung Anies Baswedan, Sia...  

[2565 rows x 6 columns]

In [5]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

def remove(tweet):
    # RT
    tweet = re.sub(r'RT[\s]+', '', tweet)
    # URLs
    tweet = re.sub(r'https\S+', '', tweet)
    tweet = re.sub(r'http[s]?\:\/\/.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', tweet)
    tweet = re.sub(r'pic.twitter.com?.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', tweet)
    # Numbers
    tweet = re.sub(r'[0-9]+', '', tweet)
    tweet = re.sub("\W+", " ", tweet)
    tweet = re.sub(r'\\n', " ", tweet)
    # Simbol
    tweet = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', tweet)
    # Koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
    tweet = re.sub(r'([a-zA-Z])\1\1','\\1', tweet)
    # Emoji
    tweet = get_emoji_regexp().sub("", tweet).strip()
    # dibawah 3 huruf
    tweet = re.sub(r'\b\w{1,3}\b', '', tweet)
    # Spasi ganda (atau lebih) menjadi satu spasi
    tweet = re.sub(' +', ' ', tweet)
    return tweet

def remove_whitespace(tweet):
    return ' '.join(tweet.split())

def tokenizing (tweet):
    tokens = word_tokenize(str.remove(tweet))
    return ' ', join(tokens)

df['cleansing'] = df['cleansing'].apply(lambda x: remove(x))

In [6]:
df.head()

id                 date      place      screen_name  \
0  1661196530127351808  2023-05-24 02:24:50  INDONESIA         jijindut   
1  1661196491896266752  2023-05-24 02:24:41        NaN  georgiussoetjip   
2  1661196455691046912  2023-05-24 02:24:33        NaN     003whitesky1   
3  1661196446203523073  2023-05-24 02:24:30        NaN          et_fath   
4  1661196388963856390  2023-05-24 02:24:17        NaN          et_fath   

                                                text  \
0  RT @eddy_widodo_165: Alhamdulilah 🤲\nBacapres ...   
1  RT @renandabachtar: Din Syamsuddin Lebih Senan...   
2  RT @PutraErlangga95: Ingin menyelamatkan Indon...   
3  RT @idtodayco: Kurang Bukti, Relawan Ganjar Pr...   
4  RT @Yurissa_Samosir: Sambutan hangat dari Masy...   

                                           cleansing  
0  Alhamdulilah Bacapres Menurut Litbang Kompas r...  
1   Syamsuddin Lebih Senang Jadi Cawapres Anies B...  
2  Ingin menyelamatkan Indonesia silahkan pilih A...  
3  Kurang Bukti Relawan Ganjar Pranowo Gagal Poli...  
4  Sambutan hangat dari Masyarakat Ponorogo Jawa ...

# CASE FOLD

In [7]:
def uppercase():
    data = df['cleansing'].str.lower()
    return data

df['case fold'] = uppercase()
results = df['case fold']
print(results)

0       alhamdulilah bacapres menurut litbang kompas r...
1        syamsuddin lebih senang jadi cawapres anies b...
2       ingin menyelamatkan indonesia silahkan pilih a...
3       kurang bukti relawan ganjar pranowo gagal poli...
4       sambutan hangat dari masyarakat ponorogo jawa ...
                              ...                        
2560    untuk bisa menjadi pemimpin kelas dunia pernya...
2561    terimakasih masyarakat brebes jawa tengah sema...
2562    anies baswedan tegaskan tidak akan lanjutkan p...
2563    petani brebes jateng deklarasi dukung anies ba...
2564    partai masyumi dukung anies baswedan siap pera...
Name: case fold, Length: 2565, dtype: object


# STOPWORDS

In [8]:
# memanfaatkan modul stopwords NLTK untuk menggunakan stopwords kustom
nltk.download('stopwords')
stopword_nltk = stopwords.words('indonesian')
more_stopword = ['anies', 'baswedan', 'ganjar', 'pranowo', 'prabowo', 'subianto']

# Sastrawi
factory = StopWordRemoverFactory()
# stopword = factory.create_stop_word_remover()
stopwords = factory.get_stop_words()+stopword_nltk+more_stopword
print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wols88\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def menghapusKata(tweet):
    clean_list = [word for word in tweet.split() if word not in stopwords]
    return clean_list

tweet_hapus = results.apply(menghapusKata)
df['stopwords'] = tweet_hapus
print(tweet_hapus)

0       [alhamdulilah, bacapres, litbang, kompas, rili...
1       [syamsuddin, senang, cawapres, ketimbang, khof...
2       [menyelamatkan, indonesia, silahkan, pilih, se...
3       [bukti, relawan, gagal, polisikan, pembangunan...
4       [sambutan, hangat, masyarakat, ponorogo, jawa,...
                              ...                        
2560    [pemimpin, kelas, dunia, pernyataan, posisi, i...
2561    [terimakasih, masyarakat, brebes, jawa, semang...
2562    [tegaskan, lanjutkan, program, jokowi, menang,...
2563    [petani, brebes, jateng, deklarasi, dukung, ca...
2564    [partai, masyumi, dukung, perang, melawan, kec...
Name: case fold, Length: 2565, dtype: object


# STEMMING

In [10]:
factory = StemmerFactory()
stem_say = factory.create_stemmer()

def stemmed_wrapper(term):
    return stem_say.stem(term)

term_base = {}

for document in tweet_hapus:
    for term in document:
        if term not in term_base:
            term_base[term] = " "
            
print(len(term_base))
print("--------------")

for term in term_base:
    term_base[term] = stemmed_wrapper(term)
    print(term,":", term_base[term])
    
print(term_base)
print("-------------")

def get_stemmed_term(document):
    return [term_base[term] for term in document]

stem_done = tweet_hapus.apply(get_stemmed_term)

print(stem_done)

5145
--------------
alhamdulilah : alhamdulilah
bacapres : bacapres
litbang : litbang
kompas : kompas
rilis : rilis
hasil : hasil
survei : survei
terbaru : baru
elektabilitas : elektabilitas
capr : capr
syamsuddin : syamsuddin
senang : senang
cawapres : cawapres
ketimbang : ketimbang
khofifah : khofifah
menyelamatkan : selamat
indonesia : indonesia
silahkan : silah
pilih : pilih
setuju : tuju
bukti : bukti
relawan : rawan
gagal : gagal
polisikan : polisi
pembangunan : bangun
jalan : jalan
jokowi : jokowi
sambutan : sambut
hangat : hangat
masyarakat : masyarakat
ponorogo : ponorogo
jawa : jawa
timur : timur
kedatangan : datang
manis : manis
buah : buah
duren : duren
mangga : mangga
muda : muda
klaten : klaten
orangnya : orang
keren : keren
pantaslah : pantas
presiden : presiden
assee : assee
ribuan : ribu
habaib : habaib
ulama : ulama
tokoh : tokoh
madura : madura
restu : restu
lantunkan : lantun
rasyid : rasyid
indah : indah
sabtu : sabtu
mengajak : ajak
terdekat : dekat
barisan : bari

menjegal : jegal
kuatkan : kuat
sukseskan : sukses
puji : puji
tuhan : tuhan
istriku : istri
tersenyum : senyum
stlh : stlh
bbrapa : bbrapa
kena : kena
mental : mental
gegara : gegara
suaminya : suami
dituduh : tuduh
baya : baya
menyerap : serap
aspirasi : aspirasi
silahturahmi : silahturahmi
kebangsaan : bangsa
pasuruan : pasuruan
negara : negara
simpatisan : simpatisan
sarankan : saran
simak : simak
dibawah : bawah
bersatu : satu
disatukan : satu
melawan : lawan
tirainya : tirai
tipis : tipis
mending : mending
antisipasi : antisipasi
kesalahan : salah
tuanmu : tuan
biar : biar
nilai : nilai
usung : usung
program : program
bantuan : bantu
operasional : operasional
ibadah : ibadah
boti : bot
apakabar : apakabar
anis : anis
preside : preside
ponpes : ponpes
darul : darul
muttaqin : muttaqin
mengusung : usung
gagasan : gagas
jiwa : jiwa
profil : profil
pahlawan : pahlawan
nasional : nasional
kakek : kakek
figur : figur
novel : novel
harkat : harkat
martabat : martabat
mata : mata
dunia :

support : support
peduli : peduli
lalunya : lalu
harap : harap
saksikan : saksi
google : google
trend : trend
purnawirawan : purnawirawan
tentara : tentara
berbicara : bicara
mendasar : dasar
berumur : umur
ratusan : ratus
gajah : gajah
mada : mada
perpustakaan : pustaka
karya : karya
taman : taman
ismail : ismail
marzuki : marzuki
mantap : mantap
pembenci : benci
sambut : sambut
pastinya : pasti
perempuan : perempuan
hebat : hebat
berkenan : kenan
balas : balas
kaos : kaos
ucapan : ucap
disoroti : sorot
pahamkan : paham
dipecat : pecat
aneh : aneh
valid : valid
membaca : baca
urusan : urus
selisih : selisih
tinggal : tinggal
capress : capress
aniesbaswedanlebihnkri : aniesbaswedanlebihnkri
umur : umur
independent : independent
kelakuan : laku
kemarin : kemarin
hoax : hoax
skrng : skrng
pssi : pssi
hina : hina
nyinyirin : nyinyirin
prinsip : prinsip
berubah : ubah
amarah : amarah
madyarakat : madyarakat
terusan : terus
dipancing : pancing
rezim : rezim
jegalaniespolitikkotorpki : jegal

jamiluddin : jamiluddin
memperkecil : kecil
langkah : langkah
wartawan : wartawan
menyudutkan : sudut
santai : santai
membe : membe
berfikir : berfikir
persaudaraan : saudara
qobul : qobul
allahuma : allahuma
saatnyabukato : saatnyabukato
maap : maap
syirik : syirik
kaga : kaga
safari : safari
cuman : cuman
rencana : rencana
agen : agen
anteknya : antek
tersandung : sandung
celahnya : celah
disingkirkan : singkir
kamerad : kamerad
dinantikan : nanti
dirindukan : rindu
diakui : aku
ajak : ajak
museum : museum
galeri : galeri
ikhlas : ikhlas
garong : garong
tabiatnya : tabiat
proses : proses
pergantian : ganti
kepemimpinan : pimpin
tahunan : tahun
estafet : estafet
rapatkan : rapat
pertempuran : tempur
ideologi : ideologi
maling : maling
dikomandoi : komando
cemen : cemen
idolanya : idola
jongos : jongos
jansen : jansen
sitindaon : sitindaon
yudhoyono : yudhoyono
kelebihan : lebih
sopan : sopan
emosi : emosi
pesta : pesta
bangkit : bangkit
mahluk : mahluk
planet : planet
menebar : tebar


prabowountukindonesia : prabowountukindonesia
neruskepakdhe : neruskepakdhe
berhadapan : hadap
berharapan : harap
teranyar : anyar
indikator : indikator
mengalami : alami
penurunan : turun
versi : versi
perse : perse
peringkat : peringkat
tertinggi : tinggi
prabo : prabo
berdasarkan : dasar
kemerosotan : merosot
nyookk : nyookk
yenny : yenny
wahid : wahid
gusdurian : gusdurian
jokowiluhutsumberpe : jokowiluhutsumberpe
jutaan : juta
tumpah : tumpah
ruah : ruah
pddibegaljokowidi : pddibegaljokowidi
dideklarasiin : dideklarasiin
bentar : bentar
kebumen : kebumen
optimistis : optimistis
dede : dede
budhyarto : budhyarto
komisaris : komisaris
pelni : pelni
dikasih : kasih
permen : permen
berpeluang : peluang
kelompok : kelompok
hiyasi : hiyasi
ruangan : ruang
sosmed : sosmed
hiroshima : hiroshima
nagasaki : nagasaki
atom : atom
kaisar : kaisar
punggawanya : punggawa
faham : faham
penti : penti
cawapre : cawapre
spontan : spontan
responden : responden
kritis : kritis
ganj : ganj
puteri : put

...

amies : amies
gass : gass
kekompakan : kompak
waspa : waspa
diantar : antar
rakyatserukanpeoplepower : rakyatserukanpeoplepower
rakyatserukanpeople : rakyatserukanpeople
kucilkan : kucil
menganggapnya : anggap
wajar : wajar
dikunjungi : kunjung
dllnya : dllnya
rizal : rizal
ramli : ramli
hidupnya : hidup
dikejar : kejar
lengser : lengser
lepas : lepas
dikontrakkan : kontrak
sekda : sekda
respons : respons
wari : wari
gasppoll : gasppoll
menlu : menlu
jepang : jepang
feat : feat
terjangkau : jangkau
menyeret : seret
bacarmol : bacarmol
tantangan : tantang
menegakkan : tegak
adilan : adil
madani : madani
sdng : sdng
kawal : kawal
mengusut : usut
rancang : rancang
mengotak : otak
cela : cela
nich : nich
dianggkap : dianggkap
dipenjara : penjara
bocoran : bocor
berinisial : inisial
santer : santer
diisukan : isu
perlawanan : lawan
mencegah : cegah
mewariskan : waris
bahaya : bahaya
penjarakan : penjara
jeddah : jeddah
kerajaan : raja
tabarakallah : tabarakallah
saudi : saudi
tabaraqallah :

berpasangan : pasang
agusyu : agusyu
bacwapres : bacwapres
panda : panda
nababan : nababan
wali : wali
menuai : tuai
bersa : bersa
demokra : demokra
jalannya : jalan
sedianya : sedia
hanta : hanta
yuda : yuda
memperoleh : oleh
sebe : sebe
kotak : kotak
adiknya : adik
wowo : wowo
julid : julid
mengajukan : aju
ditanggapi : tanggap
ditekan : tekan
tekan : tekan
berpeganglah : gang
konstitusi : konstitusi
means : means
knapa : knapa
datangi : datang
bicarakan : bicara
fotonya : foto
dicrop : dicrop
trus : trus
diganti : ganti
perhelatan : helat
black : black
widow : widow
dange : dange
sreg : sreg
khoffiah : khoffiah
tujuannya : tuju
intri : intri
segudang : gudang
kiprahnya : kiprah
diteladani : teladan
lahirkan : lahir
indeks : indeks
tetiba : tetiba
kepedulian : peduli
alas : alas
bedak : bedak
liputan : liput
main : main
jorok : jorok
bakomstra : bakomstra
pelik : pelik
plus : plus
minus : minus
slide : slide
jamuan : jamu
nampak : nampak
haus : haus
senilai : nila
diresmikan : resmi


jatidiri : jatidiri
menghabiskan : habis
trilyun : trilyun
pepsodent : pepsodent
manggis : manggis
saatnyar : saatnyar
dungu : dungu
babi : babi
bisanye : bisanye
direspon : direspon
muatan : muat
ngoceh : ngoceh
kedudukan : duduk
mengiringi : iring
pida : pida
riwayat : riwayat
pendidikannya : didik
diterimanya : terima
pinta : pinta
perangkatnya : perangkat
melupakan : lupa
cebongers : cebongers
mempersembahkan : sembah
deretan : deret
mengobrak : mengobrak
abrik : abrik
menabuh : tabuh
genderang : genderang
telur : telur
harganya : harga
mahal : mahal
rusak : rusak
letnan : letnan
ediwan : ediwan
penerima : terima
adhi : adhi
makayasa : makayasa
akademi : akademi
militer : militer
pegiat : giat
niat : niat
bntu : bntu
berpikirnya : pikir
sistimatis : sistimatis
meledak : ledak
iskan : iskan
pilihanku : pilih
penduk : penduk
mengaudit : audit
kelayakan : layak
pansus : pansus
takutnya : takut
istilah : istilah
foke : foke
menjemput : jemput
terminal : terminal
bandara : bandara
yogya

bangs : bangs
curhat : curhat
berguna : guna
persis : persis
jembut : jembut
harry : harry
tanoesoedibjo : tanoesoedibjo
pensiun : pensiun
ciptaket : ciptaket
waduuh : waduuh
kepret : kepret
makzulkanrezimmaling : makzulkanrezimmaling
makzulka : makzulka
selektif : selektif
makzulkanrezim : makzulkanrezim
pluralis : pluralis
relijius : relijius
aseng : aseng
terintegrasi : integrasi
beginilaah : beginilaah
amit : amit
intelek : intelek
kurikulumnya : kurikulum
nambah : nambah
seduh : seduh
diceritakan : cerita
ditinggal : tinggal
selundupkan : selundup
pangab : pangab
entot : entot
sopir : sopir
keharmonisan : harmonis
terjalin : jalin
kapolda : kapolda
dzalimi : dzalimi
benderang : benderang
mengandalkan : andal
feri : feri
farhati : farhati
ganis : ganis
hakimmkkudetari : hakimmkkudetari
dicap : cap
kepemimpinannya : pimpin
hakimmkk : hakimmkk
bungko : bungko
duanya : dua
pesimismu : pesimis
giring : giring
menepi : tepi
sebel : sebel
menyosialisasikan : sosialisasi
mengawal : awal
s

agustusmakzulkanjokowi : agustusmakzulkanjokowi
agustusmakzul : agustusmakzul
inipun : ini
cebong² : cebong
memframing : memframing
direken : reken
menghadang : menghadang
mall : mall
resmikan : resmi
angke : angke
fundamental : fundamental
substansial : substansial
berlanjut : lanjut
badut : badut
kacung : kacung
brebes : brebes
berbasis : bas
pengorbanan : korban
diselesaikan : selesai
relawannya : rawan
berparti : berparti
terbesar : besar
berpendidikan : didik
alimun : alimun
sofi : sofi
pemalang : malang
mong : mong
amankan : aman
keseluruh : seluruh
propinsi : propinsi
serabutan : serabutan
outsourcing : outsourcing
sebutin : sebutin
majer : majer
player : player
solusi : solusi
{'alhamdulilah': 'alhamdulilah', 'bacapres': 'bacapres', 'litbang': 'litbang', 'kompas': 'kompas', 'rilis': 'rilis', 'hasil': 'hasil', 'survei': 'survei', 'terbaru': 'baru', 'elektabilitas': 'elektabilitas', 'capr': 'capr', 'syamsuddin': 'syamsuddin', 'senang': 'senang', 'cawapres': 'cawapres', 'ketimbang

In [11]:
df['stemming'] = stem_done
print (df['stemming'])

0       [alhamdulilah, bacapres, litbang, kompas, rili...
1       [syamsuddin, senang, cawapres, ketimbang, khof...
2                [selamat, indonesia, silah, pilih, tuju]
3       [bukti, rawan, gagal, polisi, bangun, jalan, j...
4       [sambut, hangat, masyarakat, ponorogo, jawa, t...
                              ...                        
2560    [pimpin, kelas, dunia, nyata, posisi, indonesi...
2561    [terimakasih, masyarakat, brebes, jawa, semang...
2562    [tegas, lanjut, program, jokowi, menang, pilpres]
2563    [tani, brebes, jateng, deklarasi, dukung, capres]
2564    [partai, masyumi, dukung, perang, lawan, curan...
Name: stemming, Length: 2565, dtype: object


# UNTOKENIZE

In [12]:
def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()

In [13]:
df['detokens']=df['stemming'].apply(lambda x: untokenize(x))
df

id                 date                   place  \
0     1661196530127351808  2023-05-24 02:24:50               INDONESIA   
1     1661196491896266752  2023-05-24 02:24:41                     NaN   
2     1661196455691046912  2023-05-24 02:24:33                     NaN   
3     1661196446203523073  2023-05-24 02:24:30                     NaN   
4     1661196388963856390  2023-05-24 02:24:17                     NaN   
...                   ...                  ...                     ...   
2560  1686005950476677120  2023-07-31 13:28:37                     NaN   
2561  1686005857828679681  2023-07-31 13:28:15                     NaN   
2562  1686005757882593280  2023-07-31 13:27:51                     NaN   
2563  1686005743378731008  2023-07-31 13:27:48  DKI Jakarta, Indonesia   
2564  1686005736676171776  2023-07-31 13:27:46                     NaN   

          screen_name                                               text  \
0            jijindut  RT @eddy_widodo_165: Alhamdulilah 🤲\nBacapres ...   
1     georgiussoetjip  RT @renandabachtar: Din Syamsuddin Lebih Senan...   
2        003whitesky1  RT @PutraErlangga95: Ingin menyelamatkan Indon...   
3             et_fath  RT @idtodayco: Kurang Bukti, Relawan Ganjar Pr...   
4             et_fath  RT @Yurissa_Samosir: Sambutan hangat dari Masy...   
...               ...                                                ...   
2560  Joewija45450015  RT @rahmaniarbaftim: Untuk bisa menjadi Pemimp...   
2561    Ayong32516903  RT @YepryR: Terimakasih masyarakat brebes jawa...   
2562       bhozsadolf  Anies Baswedan tegaskan tidak akan lanjutkan P...   
2563      aly_benzema  RT @MissKoral: Petani Brebes Jateng Deklarasi ...   
2564       muawanah05  RT @OposisiCerdas: Partai Masyumi Dukung Anies...   

                                              cleansing  \
0     Alhamdulilah Bacapres Menurut Litbang Kompas r...   
1      Syamsuddin Lebih Senang Jadi Cawapres Anies B...   
2     Ingin menyelamatkan Indonesia silahkan pilih A...   
3     Kurang Bukti Relawan Ganjar Pranowo Gagal Poli...   
4     Sambutan hangat dari Masyarakat Ponorogo Jawa ...   
...                                                 ...   
2560  Untuk bisa menjadi Pemimpin kelas dunia pernya...   
2561  Terimakasih masyarakat brebes jawa tengah sema...   
2562  Anies Baswedan tegaskan tidak akan lanjutkan P...   
2563  Petani Brebes Jateng Deklarasi Dukung Anies Ba...   
2564  Partai Masyumi Dukung Anies Baswedan Siap Pera...   

                                              case fold  \
0     alhamdulilah bacapres menurut litbang kompas r...   
1      syamsuddin lebih senang jadi cawapres anies b...   
2     ingin menyelamatkan indonesia silahkan pilih a...   
3     kurang bukti relawan ganjar pranowo gagal poli...   
4     sambutan hangat dari masyarakat ponorogo jawa ...   
...                                                 ...   
2560  untuk bisa menjadi pemimpin kelas dunia pernya...   
2561  terimakasih masyarakat brebes jawa tengah sema...   
2562  anies baswedan tegaskan tidak akan lanjutkan p...   
2563  petani brebes jateng deklarasi dukung anies ba...   
2564  partai masyumi dukung anies baswedan siap pera...   

                                              stopwords  \
0     [alhamdulilah, bacapres, litbang, kompas, rili...   
1     [syamsuddin, senang, cawapres, ketimbang, khof...   
2     [menyelamatkan, indonesia, silahkan, pilih, se...   
3     [bukti, relawan, gagal, polisikan, pembangunan...   
4     [sambutan, hangat, masyarakat, ponorogo, jawa,...   
...                                                 ...   
2560  [pemimpin, kelas, dunia, pernyataan, posisi, i...   
2561  [terimakasih, masyarakat, brebes, jawa, semang...   
2562  [tegaskan, lanjutkan, program, jokowi, menang,...   
2563  [petani, brebes, jateng, deklarasi, dukung, ca...   
2564  [partai, masyumi, dukung, perang, melawan, kec...   

                                               stemming  \
0     [alhamdulilah, bacapres, litb

In [14]:
df.to_csv('anies_prepro.csv')